In [ ]:
# -------------------- Imports --------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

import xgboost as xgb
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, GRU, LSTM, Dense, Dropout, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# -------------------- Load Dataset --------------------
file_path = "ev_charging_patterns_augmented.csv"
if not os.path.exists(file_path):
    raise FileNotFoundError(f"CSV file not found at {file_path}")

df = pd.read_csv(file_path)
df.dropna(inplace=True)



In [3]:
# -------------------- Feature Engineering --------------------
df['Charging Start Time'] = pd.to_datetime(df['Charging Start Time'], errors='coerce')
df['Hour'] = df['Charging Start Time'].dt.hour
df['Weekday'] = df['Charging Start Time'].dt.weekday
df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

if 'Charging Power (kW)' in df.columns:
    df['Power x Duration'] = df['Charging Power (kW)'] * df['Charging Duration (hours)']
elif 'Charging Rate (kW)' in df.columns:
    df['Power x Duration'] = df['Charging Rate (kW)'] * df['Charging Duration (hours)']
else:
    df['Power x Duration'] = 0

if 'Vehicle Model' in df.columns:
    df['Model x Weekday'] = df['Vehicle Model'].astype(str) + "_" + df['Weekday'].astype(str)
else:
    df['Model x Weekday'] = df['Weekday'].astype(str)

# Lag and Rolling Features
df['Lag1_Energy'] = df['Energy Consumed (kWh)'].shift(1)
df['Lag2_Energy'] = df['Energy Consumed (kWh)'].shift(2)
df['Lag1_Duration'] = df['Charging Duration (hours)'].shift(1)
df['Rolling_Mean_Energy'] = df['Energy Consumed (kWh)'].rolling(window=3).mean()
df['Rolling_Std_Energy'] = df['Energy Consumed (kWh)'].rolling(window=3).std()
df.fillna(method='bfill', inplace=True)



C:\Users\mugal\AppData\Local\Temp\ipykernel_8384\2606197233.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [4]:
# -------------------- Data Split --------------------
targets = ['Energy Consumed (kWh)', 'Charging Duration (hours)']
drop_cols = ['User ID', 'Charging Start Time'] if 'User ID' in df.columns else ['Charging Start Time']
X = df.drop(targets + drop_cols, axis=1)
y = df[targets]

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    X[col] = LabelEncoder().fit_transform(X[col])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_rnn = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))



In [5]:
# -------------------- Evaluate Helper --------------------
def evaluate_model(name, preds):
    results = {}
    results['Model'] = name
    for i, target in enumerate(['Energy Consumed (kWh)', 'Charging Duration (hours)']):
        true = y_test.iloc[:, i]
        pred = preds[i]
        results[f"{target.split()[0]} MAE"] = mean_absolute_error(true, pred)
        results[f"{target.split()[0]} RMSE"] = np.sqrt(mean_squared_error(true, pred))
        results[f"{target.split()[0]} R2"] = r2_score(true, pred)
        results[f"{target.split()[0]} MAPE"] = mean_absolute_percentage_error(true, pred)
    return results



In [6]:
# -------------------- Model 1: GRU + XGBoost --------------------
def build_gru(input_shape):
    inp = Input(shape=input_shape)
    x = GRU(64)(inp)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(2)(x)
    model = Model(inp, out)
    model.compile(optimizer=Adam(0.0005), loss='mse')
    return model

gru_model = build_gru((X_train_rnn.shape[1], X_train_rnn.shape[2]))
gru_model.fit(X_train_rnn, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1,
              callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)

gru_train_pred = scaler_y.inverse_transform(gru_model.predict(X_train_rnn))
gru_test_pred = scaler_y.inverse_transform(gru_model.predict(X_test_rnn))

X_train_xgb = np.hstack([X_train_scaled, gru_train_pred])
X_test_xgb = np.hstack([X_test_scaled, gru_test_pred])

gru_xgboost_preds = []
for i in range(2):
    model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.03, max_depth=5,
                             subsample=0.9, colsample_bytree=0.8, random_state=42)
    model.fit(X_train_xgb, y_train.iloc[:, i])
    preds = model.predict(X_test_xgb)
    gru_xgboost_preds.append(preds)



224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step


In [7]:
# -------------------- Model 2: ANN + GRU --------------------
ann_gru_model = Sequential([
    GRU(64, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Dense(32, activation='relu'),
    Dense(2)
])
ann_gru_model.compile(optimizer=Adam(0.001), loss='mse')
ann_gru_model.fit(X_train_rnn, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1,
                  callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
ann_gru_preds_scaled = ann_gru_model.predict(X_test_rnn)
ann_gru_preds = scaler_y.inverse_transform(ann_gru_preds_scaled)
ann_gru = [ann_gru_preds[:, 0], ann_gru_preds[:, 1]]



e:\Projects\EV-Energy-Forecasting-Smart-Charging-Scheduler-main\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [8]:
# -------------------- Model 3: CNN + LSTM --------------------
cnn_lstm_model = Sequential([
    Conv1D(64, 1, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    LSTM(64),
    Dense(2)
])
cnn_lstm_model.compile(optimizer=Adam(0.001), loss='mse')
cnn_lstm_model.fit(X_train_rnn, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1,
                   callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
cnn_lstm_preds = scaler_y.inverse_transform(cnn_lstm_model.predict(X_test_rnn))
cnn_lstm = [cnn_lstm_preds[:, 0], cnn_lstm_preds[:, 1]]



e:\Projects\EV-Energy-Forecasting-Smart-Charging-Scheduler-main\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [9]:
# -------------------- Model 4: GRU + LSTM --------------------
gru_lstm_model = Sequential([
    GRU(64, return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    LSTM(64),
    Dense(2)
])
gru_lstm_model.compile(optimizer=Adam(0.001), loss='mse')
gru_lstm_model.fit(X_train_rnn, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1,
                   callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
gru_lstm_preds = scaler_y.inverse_transform(gru_lstm_model.predict(X_test_rnn))
gru_lstm = [gru_lstm_preds[:, 0], gru_lstm_preds[:, 1]]



e:\Projects\EV-Energy-Forecasting-Smart-Charging-Scheduler-main\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [10]:
# -------------------- Model 5: ANN + LSTM --------------------
ann_lstm_model = Sequential([
    LSTM(64, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Dense(32, activation='relu'),
    Dense(2)
])
ann_lstm_model.compile(optimizer=Adam(0.001), loss='mse')
ann_lstm_model.fit(X_train_rnn, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1,
                   callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
ann_lstm_preds = scaler_y.inverse_transform(ann_lstm_model.predict(X_test_rnn))
ann_lstm = [ann_lstm_preds[:, 0], ann_lstm_preds[:, 1]]




e:\Projects\EV-Energy-Forecasting-Smart-Charging-Scheduler-main\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [11]:
# -------------------- Model 6: CNN + GRU --------------------
cnn_gru_model = Sequential([
    Conv1D(64, 1, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    GRU(64),
    Dense(2)
])
cnn_gru_model.compile(optimizer=Adam(0.001), loss='mse')
cnn_gru_model.fit(X_train_rnn, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1,
                  callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
cnn_gru_preds = scaler_y.inverse_transform(cnn_gru_model.predict(X_test_rnn))
cnn_gru = [cnn_gru_preds[:, 0], cnn_gru_preds[:, 1]]



e:\Projects\EV-Energy-Forecasting-Smart-Charging-Scheduler-main\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [12]:
# -------------------- Run All Models --------------------
results = []
results.append(evaluate_model("ANN + GRU", ann_gru))
results.append(evaluate_model("CNN + LSTM", cnn_lstm))
results.append(evaluate_model("GRU + LSTM", gru_lstm))
results.append(evaluate_model("ANN + LSTM", ann_lstm))
results.append(evaluate_model("CNN + GRU", cnn_gru))
results.append(evaluate_model("GRU + XGBoost", gru_xgboost_preds))

In [14]:
# -------------------- Pretty Print Table --------------------
from tabulate import tabulate
results_df = pd.DataFrame(results)
print("\n📊 Final Model Evaluation Results (Formatted Table):\n")
print(tabulate(results_df.round(4), headers='keys', tablefmt='fancy_grid'))



📊 Final Model Evaluation Results (Formatted Table):

╒════╤═══════════════╤══════════════╤═══════════════╤═════════════╤═══════════════╤════════════════╤═════════════════╤═══════════════╤═════════════════╕
│    │ Model         │   Energy MAE │   Energy RMSE │   Energy R2 │   Energy MAPE │   Charging MAE │   Charging RMSE │   Charging R2 │   Charging MAPE │
╞════╪═══════════════╪══════════════╪═══════════════╪═════════════╪═══════════════╪════════════════╪═════════════════╪═══════════════╪═════════════════╡
│  0 │ ANN + GRU     │       0.3857 │        0.5995 │      0.9993 │        0.0528 │         0.0285 │          0.0484 │        0.9978 │          0.0189 │
├────┼───────────────┼──────────────┼───────────────┼─────────────┼───────────────┼────────────────┼─────────────────┼───────────────┼─────────────────┤
│  1 │ CNN + LSTM    │       0.3621 │        0.463  │      0.9996 │        0.0488 │         0.0323 │          0.0569 │        0.997  │          0.0192 │
├────┼───────────────┼──────